In [ ]:
!nvidia-smi

Wed Dec  8 18:04:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.1 MB 7.2 MB/s 
     |████████████████████████████████| 3.3 MB 57.3 MB/s 
     |████████████████████████████████| 596 kB 64.2 MB/s 
     |████████████████████████████████| 61 kB 688 kB/s 
     |████████████████████████████████| 895 kB 74.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 298 kB 7.9 MB/s 
     |████████████████████████████████| 1.1 MB 31.0 MB/s 
     |████████████████████████████████| 132 kB 70.5 MB/s 
     |████████████████████████████████| 243 kB 72.4 MB/s 
     |████████████████████████████████| 160 kB 72.9 MB/s 
     |████████████████████████████████| 271 kB 70.3 MB/s 
     |████████████████████████████████| 192 kB 74.0 MB/s 


In [ ]:
!unzip drive/MyDrive/ag_news_kaggle.zip

Archive:  drive/MyDrive/ag_news_kaggle.zip
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
import re

def clean(sent):
    splitsent = sent.split()
    for i in range(min(10, len(splitsent))):
        if splitsent[i] == '-' or splitsent[i] == '--':
            splitsent = splitsent[i+1:]
            break
    sent = ' '.join(splitsent)
    sent = sent.replace(' #39;', '\'')
    sent = sent.replace(' #36;', '')
    sent = sent.replace(' quot;', '\"')
    sent = sent.replace('\\', ' ')
    sent = re.sub('&lt;.*?&gt;', '', sent)
    return sent

In [ ]:
num_prompts = 5

with open("sents.txt", "w") as f:
    for t, l in zip(train.Description.tolist(), train["Class Index"].tolist()):
        sent = ' '.join([f"[PROMPT_{l}_{i}]" for i in range(num_prompts)]) + " " + clean(t)
        f.write(sent+'\n')

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, AutoModelWithLMHead, get_linear_schedule_with_warmup

import time
from tqdm.notebook import tqdm

device = "cuda"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('gpt2-medium')

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
special_tokens_dict = {'additional_special_tokens': [f"[PROMPT_{i+1}_{j}]" for i in range(4) for j in range(num_prompts)]}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = AutoModelWithLMHead.from_pretrained('gpt2-medium')

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:698: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [ ]:
model.resize_token_embeddings(len(tokenizer))
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50277, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

In [ ]:
# Freeze all params in the main transformer
for param in model.transformer.wpe.parameters():
    param.requires_grad = True
for param in model.transformer.h.parameters():
    param.requires_grad = False
for param in model.transformer.ln_f.parameters():
    param.requires_grad = False
for param in model.lm_head.parameters():
    param.requires_grad = True

In [ ]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./sents.txt",
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_gpu_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 120000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 75000
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,63.492800
1000,45.458300
1500,33.637700
2000,23.748800
2500,19.182300
3000,16.370200
3500,14.565500
4000,12.871900
4500,11.448900
5000,9.988500


Saving model checkpoint to ./checkpoint-10000
Configuration saved in ./checkpoint-10000/config.json
Model weights saved in ./checkpoint-10000/pytorch_model.bin
Saving model checkpoint to ./checkpoint-20000
Configuration saved in ./checkpoint-20000/config.json
Model weights saved in ./checkpoint-20000/pytorch_model.bin
Saving model checkpoint to ./checkpoint-30000
Configuration saved in ./checkpoint-30000/config.json
Model weights saved in ./checkpoint-30000/pytorch_model.bin
Deleting older checkpoint [checkpoint-10000] due to args.save_total_limit
Saving model checkpoint to ./checkpoint-40000
Configuration saved in ./checkpoint-40000/config.json
Model weights saved in ./checkpoint-40000/pytorch_model.bin
Deleting older checkpoint [checkpoint-20000] due to args.save_total_limit
Saving model checkpoint to ./checkpoint-50000
Configuration saved in ./checkpoint-50000/config.json
Model weights saved in ./checkpoint-50000/pytorch_model.bin
Deleting older checkpoint [checkpoint-30000] due to 

CPU times: user 3h 24min 40s, sys: 16min 44s, total: 3h 41min 24s
Wall time: 3h 41min 49s


TrainOutput(global_step=75000, training_loss=4.158038642578125, metrics={'train_runtime': 13309.0456, 'train_samples_per_second': 45.082, 'train_steps_per_second': 5.635, 'total_flos': 7.042742222064845e+16, 'train_loss': 4.158038642578125, 'epoch': 5.0})

# Validation

In [ ]:
mod = AutoModelWithLMHead.from_pretrained('./drive/MyDrive/checkpoint-60000')
mod.to(device)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:698: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50277, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

In [ ]:
sent = "Lebron James scored twenty points"
print(sent)
sent0 = ' '.join([f"[PROMPT_0_{i}]" for i in range(5)]) + " " + sent[:300]
sent1 = ' '.join([f"[PROMPT_1_{i}]" for i in range(5)]) + " " + sent[:300]   # world
sent2 = ' '.join([f"[PROMPT_2_{i}]" for i in range(5)]) + " " + sent[:300]   # sports
sent3 = ' '.join([f"[PROMPT_3_{i}]" for i in range(5)]) + " " + sent[:300]   # business
sent4 = ' '.join([f"[PROMPT_4_{i}]" for i in range(5)]) + " " + sent[:300]   # sci and tech
for sent in [sent1, sent2, sent3, sent4]:
    print(sent)
    input_ids = torch.tensor(tokenizer.encode(sent)).to(device)
    greedy_output = mod.generate(torch.tensor(input_ids).unsqueeze(0), do_sample=True, max_length=100, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=3,
    no_repeat_ngram_size=3)
    print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Lebron James scored twenty points
[PROMPT_1_0] [PROMPT_1_1] [PROMPT_1_2] [PROMPT_1_3] [PROMPT_1_4] Lebron James scored twenty points


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


     Lebron James scored twenty points and the Chicago Bulls beat the Atlanta Hawks 102-101 on Sunday night, extending their winning streak to five games. The Bulls continued their winning ways by beating the Washington Wizards 94-93. Their record was 21-8 with James, who hit a pair of threes with 3:23 left to cap the game. James finished with 33 points, seven rebounds, three assists and two steals. After the game, Bulls
[PROMPT_2_0] [PROMPT_2_1] [PROMPT_2_2] [PROMPT_2_3] [PROMPT_2_4] Lebron James scored twenty points


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


     Lebron James scored twenty points and No. 10 Miami beat Connecticut 70-59 Saturday night to take a one-shot lead in the best-of-three Eastern Conference finals. With Miami up two games to none, James scored 20 points and added nine rebounds and two assists to guide the Heat to an 80-58 victory over their archrival, which moved to the final for the fifth consecutive year. Miami has won at least three of the last
[PROMPT_3_0] [PROMPT_3_1] [PROMPT_3_2] [PROMPT_3_3] [PROMPT_3_4] Lebron James scored twenty points


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


     Lebron James scored twenty points in Boston yesterday, but he didn't win, losing the Cleveland Cavaliers 108-79. The game was played at Harvard Square, a popular gathering place for Boston residents. James had to go out of the way to get a fan after the game because the crowd at HarvardSquare wasn't interested in him being in the area. He had a few shots, but no one bothered to do a search on his name on the
[PROMPT_4_0] [PROMPT_4_1] [PROMPT_4_2] [PROMPT_4_3] [PROMPT_4_4] Lebron James scored twenty points
     Lebron James scored twenty points, but the Cleveland Cavaliers did not get anything from the Indiana Pacers in the preseason. One of the Cavaliers' most promising plays to date is the dunk he made to put the Cavaliers ahead and help carry a late rally to a 111-109 win over the Indiana Hoosiers. Cleveland won 95-92 at Palace of Auburn Hills and is back in action tonight to face the New York Knicks, while the Indiana Fever play


In [ ]:
idx = 0
for sent in df.text.tolist():
    if idx <= 88:
        idx += 1
        continue
    sent = "The school"
    print(sent)
    sent0 = ' '.join([f"[PROMPT_0_{i}]" for i in range(5)]) + " " + sent[:20]
    sent1 = ' '.join([f"[PROMPT_1_{i}]" for i in range(5)]) + " " + sent[:20]   # world
    sent2 = ' '.join([f"[PROMPT_2_{i}]" for i in range(5)]) + " " + sent[:20]   # sports
    sent3 = ' '.join([f"[PROMPT_3_{i}]" for i in range(5)]) + " " + sent[:20]   # business
    sent4 = ' '.join([f"[PROMPT_4_{i}]" for i in range(5)]) + " " + sent[:20]   # sci and tech
    for sent in [sent1, sent2, sent3, sent4]:
        print(sent)
        input_ids = torch.tensor(tokenizer.encode(sent)).to(device)
        greedy_output = mod.generate(torch.tensor(input_ids).unsqueeze(0), do_sample=True, max_length=100, 
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=3,
        no_repeat_ngram_size=3)
        print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))
    break

NameError: ignored

# Generate Test Predictions

In [ ]:
ins = []
for i in test.Description.tolist():
    ins.append(clean(i))

In [ ]:
world2, sports2, business2, scitech2 = [], [], [], []
test_final = pd.read_csv("test_final.csv")

for ii, i in enumerate(test_final.Phrase.tolist()):
    i = ' '.join(i.split()[:3])
    sents = [' '.join([f"[PROMPT_{j+1}_{i}]" for i in range(5)]) + " " + i for j in range(4)]
    for idx, sent in enumerate(sents):
        input_ids = torch.tensor(tokenizer.encode(sent)).to(device)
        greedy_output = mod.generate(torch.tensor(input_ids).unsqueeze(0), do_sample=True, max_length=100, 
            top_k=50, 
            top_p=0.95, 
            num_return_sequences=3,
            no_repeat_ngram_size=3)
        pred = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
        print(ii, idx, pred)
        if idx==0: world2.append(pred)
        if idx==1: sports2.append(pred)
        if idx==2: business2.append(pred)
        if idx==3: scitech2.append(pred)
    print()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0 0      Pixar Animation Studio Inc. on Sunday said it has hired its first-ever chief executive, a man with extensive marketing and public relations skills. Pixar's new chief executive will be Jeff Williams, 48, who worked for the entertainment company in its early days and who is also the first director to join Pixar's ranks. Williams' hiring brings to five the number of Pixar insiders, and brings to three the number currently at Pixar. In 2005, the company added


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0 1      Pixar Animation Studio President Alan Horn said yesterday that the studio's forthcoming feature "Finding Nemo" will include the first official footage of a previously underwater movie. With new underwater camera technology,"Finding Nem" footage could be coming in its final product, after the feature's director, John Lasseter, will reportedly be able to take the footage to the surface. "    " The film will be set in an underwater world called " 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

199 0      But as far as we know, his son was not in Nigeria at the time of his slaying. Some reports say that his car crashed in the Niger Delta. On Monday, Nigeria's ruling party said the man who killed two Frenchmen two weeks ago was a member of its own elite military. A senior official said the military was not directly involved. "But at the beginning of January I asked them to look into whether it was true," the official said


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


199 1      But as far as a new stadium goers are concerned, this one's no longer the marquee play for the city. That honor has moved to Yankee Stadium as the Yankees face a host of challenges in getting ready to start the season at home, not least their new home-field advantage in the AL East. Meanwhile, the Red Sox, who began the 2004 season with a 10-game lead over the Yankees for first place in baseball, are in the


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


199 2      But as far as Wall Street is concerned, it wasn't the worst of Monday's stock market rally. NEW YORK (CNN/Money) - The first time a U.S. president's stock gains more than 5 percent in a year - that's what the Dow Jones Industrial Average was doing. The market was up 5...w5.10...6...w7...w9...w16...w21...w23...w29...
199 3      But as far as new games are concerned, it seems the Xbox Live community is more worried about Microsoft than Sony. While some gamers are happy to let others play with them, others are just fine with people sharing online accounts and using them for free games. Many feel that Microsoft has done enough in the past four or five months to get the community going again. -washingtonpost.com. The Washington Post. A new day is dawning for gamers



In [ ]:
num = 3

print(world2[num])
print(sports2[num])
print(business2[num])
print(scitech2[num])

     While manager Joe Torre's team struggled with a 10-game losing streak, he was still trying to decide if he should give up his three-game suspension. Torre, who has not been disciplined for his four games, said his decision had not been discussed with his players before Thursday's game, when he announced the suspension. He said, "I really believe that it's in the best interest of my players... for me to give... the
     While manager Joe Torre looked to salvage an 8-7 record and a pair of division crowns for the Yankees, the pitching coach took another gamble on a rookie, who, at age 19, already was just 24. How long will his promising career last? How big of a lead did Torre have to give up in order to move on to a bigger pitcher in the rotation? Torre turned down the three-year, $25 million contract
     While manager Joe Torre spent the last two seasons as the San Francisco Chronicle baseball page's managing editor, the managing editor of the San Diego Union-Tin Daily newspaper 

In [ ]:
i = test_final.Phrase.tolist()[num]
i = ' '.join(i.split()[:3])
i = "The potato"
sents = [' '.join([f"[PROMPT_{j+1}_{i}]" for i in range(5)]) + " " + i for j in range(4)]
for idx, sent in enumerate(sents):
    input_ids = torch.tensor(tokenizer.encode(sent)).to(device)
    greedy_output = mod.generate(torch.tensor(input_ids).unsqueeze(0), do_sample=True, max_length=100, 
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=3,
        no_repeat_ngram_size=3)
    pred = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
    print(pred)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


     The potato-rich country of Namibia had an Olympic swimming competition yesterday and yesterday was in the news for a similar reason. The International Olympic Committee have pulled out all the stops in their attempts to stop the African countries from holding the 2006 Winter Games in Baku. The Games are to be held in the Azerbaijan capital, Baku, from January 2006 to 2008. There is a high likelihood, said the IOC, that a high number of local police


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


     The potato was once synonymous with potatoes, but only because of the "pipeline" of other vegetables and sprouts that are grown throughout the country. It is now being pushed out of the spotlight because of soybean cultivation, according to a report released Wednesday by the Food and Drug Administration.... According to the study, potato plants lost  $41 billion between 1995 and 2007 due to soybean-related pollution.... Now, the study says,


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


     The potato chip industry will be a little easier to grow if genetically altered plants can escape from fields and are picked up later by farmers. The move could lead to a faster transition away of genetically engineered crops. By Peter G. Srichler, The Associated Press. By Tom O'Brien, The AP. By Dan Gilkey and Martha Grossman, AP./USATSI.com. By Michaela Delacroix, The Washington
     The potato plant will be used as a replacement for rice in Japan's current and next-generation versions of the nation's staple dish. The new "   $1.28 billion" project, announced Friday by the Japanese government, will produce 3,200 new rice varieties from a new mix of potato and brown rice. The project's two major component parts will be the development of the technology for making the brown and potato varieties and the creation of a


In [ ]:
test_final["world"] = world2
test_final["sports"] = sports2
test_final["business"] = business2
test_final["scitech"] = scitech2
test_final["world3"] = world3
test_final["sports3"] = sports3
test_final["business3"] = business3
test_final["scitech3"] = scitech3
test_final.to_csv("test_final2.csv", index=False)

In [ ]:
newmod = AutoModelWithLMHead.from_pretrained('gpt2-medium')
newmod.to(device)
newtok = AutoTokenizer.from_pretrained('gpt2-medium')

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:698: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [ ]:
world3, sports3, business3, scitech3 = [], [], [], []
test_final = pd.read_csv("test_final_v3.csv")

for ii, i in enumerate(test_final.Phrase.tolist()):
    i = ' '.join(i.split()[:3])
    sents = [f"This is a sentence about {j}. " + i for j in ["world politics", "sports", "business", "science and technology"]]
    for idx, sent in enumerate(sents):
        input_ids = torch.tensor(newtok.encode(sent)).to(device)
        greedy_output = newmod.generate(torch.tensor(input_ids).unsqueeze(0), do_sample=True, max_length=100, 
            top_k=50)
        pred = newtok.decode(greedy_output[0], skip_special_tokens=True)
        print(ii, idx, pred)
        if idx==0: world3.append(pred)
        if idx==1: sports3.append(pred)
        if idx==2: business3.append(pred)
        if idx==3: scitech3.append(pred)
    print()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0 0 This is a sentence about world politics. Carly Patterson upstaged me during one of the many debates between Bernie Sanders and Donald Trump, when he was asked: "How will the economy perform after you're elected as the next president?" I didn't even know the world was going to elect someone who has no qualifications other than being a world expert in the subject at hand. It is impossible to grasp the scale of that question.

Sanders didn't respond to the question, and I guess we


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0 1 This is a sentence about sports. Carly Patterson upstaged her two friends before they started throwing punches, and there was more to it than that.

Patterson is a junior at St. Mary's. She and two other friends are the first members of her high school basketball team to ever play at the collegiate level. She's also the only one of her four teammates to make the AAU circuit. She's ranked a top three, one of the best prospects for Kentucky, and one


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

74 0 This is a sentence about world politics. A great man in great times, whose death was precipitated by misfortune and who was then hailed across the world because he had been able to use this tragedy to bring about the transformation of the political situation of the world.

So, we have two sets of texts of this very great man, whose life is on the line of history. They give us the context of his life, but we cannot read these texts without knowing the time and place of his


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


74 1 This is a sentence about sports. A great man with a great wife, two great kids, an incredible band, and a good job. Just like all that.

I love being able to bring that same joy back to my readers every day. You know what to do! Give me a shout. Send me a tweet to say hello, say goodbye, show me how good you all are. And to know that you know me. I hope you stick around for plenty more of this.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


74 2 This is a sentence about business. A great man was never someone with money before making his first contribution…that is not money…

To be honest, it also felt like a strange statement to make. In my case, my entire life was built on money and, in the end, that's been my dream to give back to my community…

"A man and wife should be able to afford to live together by themselves."

That was my thought once I finished reading that
74 3 This is a sentence about science and technology. A great man died before he had time to write something. And he wrote a great book called Life and Death. We're talking long life, great achievements, and people die of natural causes. It's just incredible. In fact, scientists would call this an example of evolution. It's basically a scientific explanation of the laws of nature, but it doesn't claim to be more scientific than other explanations. It's just one more attempt to get some science



In [ ]:
# Copyright (c) Tsinghua university conversational AI group (THU-coai).
'''
[1] Li, Jiwei, et al. "A diversity-promoting objective function for neural conversation models."
    arXiv preprint arXiv:1510.03055 (2015).
'''

import math
import sys
import json
import torch
from transformers import OpenAIGPTLMHeadModel, OpenAIGPTTokenizer, GPT2Tokenizer, GPT2LMHeadModel

#tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
#model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

model.eval()

def perplexity(sent):
    indexed_tokens = tokenizer.encode(sent)
    tokens_tensor = torch.tensor([indexed_tokens])
    with torch.no_grad():
        outputs = model.forward(tokens_tensor, labels=tokens_tensor)
    loss = outputs[0]
    return math.exp(loss.item())


def read_dialog(file):
    """
    Read dialogs from file
    :param file: str, file path to the dataset
    :return: list, a list of dialogue (context) contained in file
    """
    with open(file) as f:
        contents = [i.strip() for i in f.readlines() if len(i.strip()) != 0]
    return [json.loads(i) for i in contents]


def count_ngram(hyps_resp, n):
    """
    Count the number of unique n-grams
    :param hyps_resp: list, a list of responses
    :param n: int, n-gram
    :return: the number of unique n-grams in hyps_resp
    """
    if len(hyps_resp) == 0:
        print("ERROR, eval_distinct get empty input")
        return

    if type(hyps_resp[0]) != list:
        print("ERROR, eval_distinct takes in a list of <class 'list'>, get a list of {} instead".format(
            type(hyps_resp[0])))
        return

    ngram = set()
    for resp in hyps_resp:
        if len(resp) < n:
            continue
        for i in range(len(resp) - n + 1):
            ngram.add(' '.join(resp[i: i + n]))
    return len(ngram)


def eval_distinct(hyps_resp):
    """
    compute distinct score for the hyps_resp
    :param hyps_resp: list, a list of hyps responses
    :return: average distinct score for 1, 2-gram
    """

    hyps_resp = [list(map(str, tokenizer.encode(h))) for h in hyps_resp]

    if len(hyps_resp) == 0:
        print("ERROR, eval_distinct get empty input")
        return

    if type(hyps_resp[0]) != list:
        print("ERROR, eval_distinct takes in a list of <class 'list'>, get a list of {} instead".format(
            type(hyps_resp[0])))
        return

    hyps_resp = [(' '.join(i)).split() for i in hyps_resp]
    num_tokens = sum([len(i) for i in hyps_resp])
    dist1 = count_ngram(hyps_resp, 1) / float(num_tokens)
    dist2 = count_ngram(hyps_resp, 2) / float(num_tokens)
    dist3 = count_ngram(hyps_resp, 3) / float(num_tokens)

    return dist1, dist2, dist3

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [ ]:
test_final["world4"] = world3
test_final["sports4"] = sports3
test_final["business4"] = business3 
test_final["scitech4"] = scitech3
test_final.to_csv("test_final_v4.csv", index=False)

In [ ]:
from tqdm.notebook import tqdm
a = sports3
ppls = [perplexity(i) for i in tqdm(a)]
print(c)
print(eval_distinct(a))
print(sum(ppls)/len(ppls))

  0%|          | 0/75 [00:00<?, ?it/s]

NameError: ignored

In [ ]:
world, sports, business, scitech = [], [], [], []

for ii, i in enumerate(ins):
    print("Currently at idx: {}/{}".format(ii, len(ins)))
    first_five_words = ' '.join(i.split()[:5])
    print(first_five_words)
    sents = [' '.join([f"[PROMPT_{j+1}_{i}]" for i in range(5)]) + " " + first_five_words for j in range(4)]
    for idx, sent in enumerate(sents):
        input_ids = torch.tensor(tokenizer.encode(sent)).to(device)
        greedy_output = mod.generate(torch.tensor(input_ids).unsqueeze(0), do_sample=True, max_length=100, 
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=3,
        no_repeat_ngram_size=3)
        
        pred = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
        if idx==0: world.append(pred)
        if idx==1: sports.append(pred)
        if idx==2: business.append(pred)
        if idx==3: scitech.append(pred)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Currently at idx: 0/7600
Unions representing workers at Turner


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Currently at idx: 1/7600
TORONTO, Canada -- A second


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Currently at idx: 1117/7600
An Argentine court acquitted five


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Currently at idx: 1118/7600
The adapters can overheat and


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: ignored

In [ ]:
for ii, x in enumerate(sports):
    print(ii, x)

0      Unions representing workers at Turner Broadcasting's broadcasting and cable operations in Virginia and Maryland are urging Gov. Bob McDonnell (R) and his cabinet to reject legislation from the governor's re-election committee that would force public workers to accept a higher pay level. The proposal, first reported by the Washington Post, was introduced Friday. By a wide margin, union representatives backed the bill Friday night, although they said at least a handful of other groups were opposed.
1      TORONTO, Canada -- A second consecutive season for a starting pitcher is always a good thing, but this time in Game 1 of the Subway Series against the Toronto Blue Jays, Jeff Tomlin might not have too many choice words. The lefty lefty has been hit with a double by Jose Bautista and he's probably not in for good. In the same game, the Blue Jays lefty Matt Bush will also take the hill against his
2      A company founded by a British company executive who worked on the BHS compute

In [ ]:
num = 140

print(world2[num])
print(sports2[num])
print(business2[num])
print(scitech2[num])

     Advance copies of a report released Monday by the State Department's inspector General into the detention and abuse of Iraqi asylum seekers show the Bush administration is in danger of missing an important deadline and could face a lawsuit. A March 25 memo sent to attorneys from the Office of Legal Counsel gave State no date, so the agency couldn't immediately say whether its legal case would move ahead or if the government had won. In the memorandum, agency lawyers say the OLC is
     Advance copies of Tiger Woods's World Match Play card are on sale for only  $25 on Tiger's online store, the PGA Tour said Monday, a 25-percent discount after an exclusive deal with Tiger's representative. The card, which was released Friday, is also available for free on the PFR service. The service includes an interview with Woods with his manager Tom Kite. Woods's PGA TOUR account is available only to members of Tiger
     Advance copies of the final book of her autobiography, "If Only," made ava

In [ ]:
[1114, 67]

In [ ]:
basetokenizer = AutoTokenizer.from_pretrained('gpt2')
# special_tokens_dict = {'additional_special_tokens': [f"[PROMPT_{i}_{j}]" for i in range(4) for j in range(num_prompts)]}
# num_added_toks = basetokenizer.add_special_tokens(special_tokens_dict)
basetokenizer.pad_token = basetokenizer.eos_token
basemodel = AutoModelWithLMHead.from_pretrained('gpt2')
# basemodel.resize_token_embeddings(len(basetokenizer))
basemodel.to(device)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:698: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [ ]:
sent = "The school"
for j in ["world politics", "sports", "business", "science and technology"]:
    input_ids = torch.tensor(basetokenizer.encode(f"This is a sentence about {j}. {sent}")).to(device)
    print(input_ids)
    greedy_output = basemodel.generate(torch.tensor(input_ids).unsqueeze(0), do_sample=True, max_length=100, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=3)
    print("Output: ", basetokenizer.decode(greedy_output[0], skip_special_tokens=True))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([1212,  318,  257, 6827,  546,  995, 4819,   13,  383, 1524],
       device='cuda:0')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:  This is a sentence about world politics. The school of thought on the topic has not changed. The words to which it refers, though, are not changed but are now used by politicians in order to advance their agendas of further development and more radical or repressive.

It is not surprising to learn that the word "radical" was the most widely used term in the 20th century. It is true that, for the most part, it was used in the name "reform." The term
tensor([1212,  318,  257, 6827,  546, 5701,   13,  383, 1524], device='cuda:0')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:  This is a sentence about sports. The school is a public school. If we're going to run an athletic program like this, we need to build the support and the facilities that I'm trying to create.

"I'm a person who lives in a very small town, where many people love to see basketball in the backyard and I understand that. But I can also make a living working a part-time job, with a job that pays well in excess of $5,000.
tensor([1212,  318,  257, 6827,  546, 1597,   13,  383, 1524], device='cuda:0')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:  This is a sentence about business. The school was asked to conduct a follow-up investigation. They found that the school was violating Title IX. The school said it was "not doing a good job of taking into account the conduct on the part of the students." They also found that "the students were verbally harassed, physically and verbally assaulted, sexually harassed, and physically attacked by a member of the staff when they were not in the classroom." And, apparently, none of that happened.


tensor([1212,  318,  257, 6827,  546, 3783,  290, 3037,   13,  383, 1524],
       device='cuda:0')
Output:  This is a sentence about science and technology. The school is focused on teaching science and technology to the rest of school, because that is what's required from all of us in America.

There is no difference whatsoever when an entire department is going to the public school or in a particular program of a particular school district. And it's absolutely vital to know that.

That's

In [ ]:

output_model = './models/my_model.pth'

# save
torch.save({'model_state_dict': model.state_dict()}, output_model)

# load
# checkpoint = torch.load(output_model, map_location='cpu')
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

RuntimeError: ignored